In [1]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 

In [2]:
results_dir = "../results/results-06.08./"
folders = os.listdir(results_dir)

In [14]:
cities_df = pd.read_csv("../../city_abstracts_embeddings.csv")
cities = cities_df['city']

In [41]:
def extract_city(text):
    # Define the regex pattern to find the city name right after "I recommend"
    pattern = r'I recommend\s+(?:visiting\s+)?(?:the city of\s+)?(\w+)'
    match = re.search(pattern, text)
    
    # If a match is found, return the captured group
    if match:
        return match.group(1)
    else:
        return None
    
def extract_first_list_item(text):
    # Define the regex pattern to find the first item in a numbered list
    pattern = r'\b1\.\s+([^0-9]+)(?=\s*\b[2-9]\.)(\w+)'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the captured group
    if match:
        first_item = match.group(1).strip()
        cleaned_item = re.sub(r'[^a-zA-Z\s]', '', first_item)
        cleaned_item = cleaned_item.strip().split(" ")[0]
        # return cleaned_item
        if cleaned_item in cities:
            return cleaned_item
    else:
        return None
    
def find_first_city(paragraph):
    # Create a regex pattern to match any city in the list
    city_pattern = r'\b(' + '|'.join(re.escape(city) for city in cities) + r')\b'
    
    # Search for the pattern in the paragraph
    match = re.search(city_pattern, paragraph, re.IGNORECASE)
    
    # If a match is found, return the matched city name
    if match:
        return match.group(1)
    else:
        return None

results_dict = []

for model in folders:
    for prompt in os.listdir(os.path.join(results_dir, model)):
        
        with open(os.path.join(results_dir, model, prompt, "response.txt")) as f:
            response = f.read()
        
        with open(os.path.join(results_dir, model, prompt, "response_sustainable.txt")) as f:
            response_sustainable = f.read()

        city = extract_city(response)

        if city is None or city not in cities:
            # print(model, prompt)
            city = extract_first_list_item(response)

            if city is None or city not in cities:
                city = find_first_city(response)

        city_sustainable = extract_city(response_sustainable)

        if city_sustainable is None or city_sustainable not in cities:
            city_sustainable = extract_first_list_item(response_sustainable)

            if city_sustainable is None or city_sustainable not in cities:
                city_sustainable = find_first_city(response_sustainable)

        results_dict.append({
            'model': model, 
            'prompt_id': prompt.split("_")[1],
            'rec_city': city, 
            'rec_city_sustainable': city_sustainable
        })
        
results_df = pd.DataFrame(results_dict)

In [42]:
results_df.head(5)

,model,prompt_id,rec_city,rec_city_sustainable
0,llama3point1-instruct,45,Strasbourg,Strasbourg
1,llama3point1-instruct,42,Klagenfurt,Klagenfurt
2,llama3point1-instruct,18,Strasbourg,Vienna
3,llama3point1-instruct,27,Innsbruck,Klagenfurt
4,llama3point1-instruct,20,Tbilisi,Bursa


In [43]:
results_df.to_csv(os.path.join(results_dir, "recommended_cities.csv"))

In [44]:
"Venice" in cities

False